# POP MOC(sigma 2) for 0.1-degree
**Input Data:** Monthly POP output timeseries files  
**Output Data:** Monthly mean AMOC sigma 2 timeseries  
**Description:** Computes MOC(sigma 2) offline from POP history files using simple xhistogram binning.  
**Date:** February 2023  
**Creator:** Steve Yeager (https://github.com/sgyeager/POP_MOC/blob/main/notebooks/pop_MOCsigma2_0.1deg.ipynb)  
**Updated:** Fred Castruccio and Teagan King, February 2023  
**Note:** To use the MOCutils, a user will need to clone the POP_MOC repository (https://github.com/sgyeager/POP_MOC) and install MOCutils by going to the POP_MOC directory and running `pip install -e . --user`.

In [1]:
%load_ext autoreload
%autoreload 2
import cftime
import copy
import dask
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from MOCutils import popmoc
import numpy as np  
import os
import pop_tools
import time
import xarray as xr 
from xhistogram.xarray import histogram

In [2]:
from dask.distributed import wait
dask.__version__

'2022.11.0'

In [3]:
pop_tools.__version__

'2023.3.0'

In [4]:
# Close out Dask Cluster and release workers:
# cluster.close()
# client.close()

In [5]:
# TODO: optimize dask resources

def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=10GB',
        account='P93300313',
        walltime='06:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        "distributed.scheduler.worker-saturation": "1.0",
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(30) 

/glade/u/home/tking/.conda/envs/ipogs_from_steve/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36542 instead
  warnings.warn(


In [6]:
client

<Client: 'tcp://10.12.206.48:33155' processes=30 threads=30, memory=300.00 GiB>

In [7]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

# Get the required variables 

In [8]:
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002/ocn/proc/tseries/month_1/'
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003/ocn/proc/tseries/month_1/'
#fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP45/HR/b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP26/HR/b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003/ocn/proc/tseries/month_1/'

#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.VVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.VVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.VVEL.200601-210012.nc'
dsV = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsV = time_set_midmonth(dsV,'time')

#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.30.002.pop.h.WVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP85C5.ne120_t12.cesm-ihesp-hires1.0.31.003.pop.h.WVEL.200601-210012.nc'
#fin = fdir + 'b.e13.BRCP45C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.WVEL.200601-210012.nc'
dsW = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsW = time_set_midmonth(dsW,'time')


fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.TEMP.200601-210012.nc'
dsT = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsT = time_set_midmonth(dsT,'time')

fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.SALT.200601-210012.nc'
dsS = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsS = time_set_midmonth(dsS,'time')

fin = fdir + 'b.e13.BRCP26C5.ne120_t12.cesm-ihesp-hires1.0.42.003.pop.h.UVEL.200601-210012.nc'
dsU = xr.open_dataset(fin, chunks={'time':1,'nlon':100})
dsU = time_set_midmonth(dsU,'time')

fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

ERROR 1: PROJ: proj_create_from_database: Open of /glade/u/home/tking/.conda/envs/ipogs_from_steve/share/proj failed


In [9]:
u_e_all = dsU['UVEL']
u_e_all = u_e_all.where(u_e_all<1.e30,0)
v_e_all = dsV['VVEL']
v_e_all = v_e_all.where(v_e_all<1.e30,0)

# Get model T & S
salt_all = dsS['SALT']
temp_all = dsT['TEMP']

tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

### MOC Region Mask

In [10]:
## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

In [11]:
# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

In [12]:
%%time
dz = ds_grid['dz'].persist() / 100.
print('got dz')
kmt = ds_grid['KMT'].fillna(0).persist() 
print('got kmt')
# Slow step (~12 mins)
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)
print('got dzt dzu')

got dz
got kmt
got dzt dzu
CPU times: user 1min, sys: 4min 29s, total: 5min 29s
Wall time: 12min 28s


In [13]:
# Compute sigma-2 field from POP model output
refz = 2000
refdep = xr.DataArray(refz)

# Grid Metrics
print('computing grid metrics at {}'.format(time.ctime()))
dxu = ds_grid['DXU']
dyu = ds_grid['DYU']
dxt = ds_grid['DXT']
dyt = ds_grid['DYT']

computing grid metrics at Mon Mar  6 16:37:03 2023


# Loop over time slices and compute MOC 

In [14]:
# ystart=[2006,2010,2020,2030,2040,2050,2060,2070,2080,2090,2100]
# yend=[2009,2019,2029,2039,2049,2059,2069,2079,2089,2099,2100]

ystart=[2006]
yend=[2006]

In [16]:
%%time
for n in range(len(ystart)):
    print("starting loop over {}-{} at {}".format(ystart[n], yend[n], time.ctime()))
    print('determining time subset of files to operate on')
    v_e = v_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    u_e = u_e_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    salt = salt_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))
    temp = temp_all.sel(time=slice(cftime.DatetimeNoLeap(ystart[n], 1, 1), cftime.DatetimeNoLeap(yend[n], 12, 31)))

    # Sigma2 on model TLAT, TLONG
    print('calculating sigma2_T: {}'.format(time.ctime()))
    sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
    sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})
    print('calculated sigma2_T: {}'.format(time.ctime()))
    
    print("sigma2_T {}".format(sigma2_T))
    
    # Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
    sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()
    print('sigma_mid {}'.format(sigma_mid))
    print('sigma_edge {}'.format(sigma_edge))

    # Compute Isopycnal Layer Thickness (Can skip if not needed)
    # Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
    iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
    iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

    # Use histogram to compute layer thickness. Vertical sum should be same as HT.
    iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzt,dim=['z_t'],density=False)
    iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_thick = iso_thick.rename('iso_thick').assign_attrs({'units':'m',
                                                            'long_name':'Isopycnal Layer Thickness'}).rename({'sigma':
                                                                                                              'sigma_mid'})
    print('iso_thick {} at {}'.format(time.ctime(), iso_thick))
    iso_thick = iso_thick.transpose('time','sigma_mid','nlat','nlon')

    # Compute Isopycnal Layer Depth (Can skip if not needed)
    # Cumulative sum of layer thickness yields depth of layer edges:
    iso_depth = iso_thick.cumsum('sigma_mid').rename('iso_depth').rename({'sigma_mid':'sigma_bot'}).assign_attrs({'units':'m','long_name':'Isopycnal Layer Depth'})
    sigma_bot = sigma_edge.isel(sigma=slice(1,None)).rename({'sigma':'sigma_bot'}).assign_attrs({'long_name':'Sigma2 at bottom of layer'})
    iso_depth['sigma_bot'] = sigma_bot
    iso_depth = iso_depth.transpose('time', 'sigma_bot','nlat','nlon')
    print('iso_depth {} at {}'.format(time.ctime(), iso_depth))
    
    # Compute Isopycnal Layer Horizontal Volume Flux
    
    # Grid-oriented Volume FLuxes:
    print('generating grid-oriented volume fluxes at {}'.format(time.ctime()))
    u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    
    # Convert u_e,v_e to C-grid fluxes
    print('converting to c-grid fluxes at {}'.format(time.ctime()))
    u = 0.5*(u_e+u_e.shift(nlat=1))
    v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))
    
    # Volume fluxes in density-space. 
    print('generating iso_flux at {}'.format(time.ctime()))
    iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
    iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
    iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    
    print('performing dask operations at {}'.format(time.ctime()))
    iso_uflux = iso_uflux.persist()
    daskout = wait(iso_uflux)
    iso_vflux = iso_vflux.persist()
    daskout = wait(iso_vflux)
 
    # Compute Vertical Volume Flux from horizontal flux convergence
    print('computing vertical volume flux at {}'.format(time.ctime()))
    wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
    wflux = wflux.assign_coords({'TLAT':tlat,'TLONG':tlon})
    wflux = wflux.persist()
    daskout = wait(wflux)
    
    # Compute MOC
    print('computing MOC at {}'.format(time.ctime()))
    MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
    MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
    MOC = dask.optimize(MOC)[0]
    MOC = MOC.load()
    print('loaded MOC')

    # add vflux at southern boundary of Atlantic domain
    tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
    tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
    moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
    moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
    MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

    # Save to netcdf
    MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
    dsout = MOCann.to_dataset()
    
    outdir = os.path.dirname(fin)
    fout = os.path.split(fin)[-1].split('.')[:-3]
    fout.append('MOCsig')
    fout.append('{:04d}-{:04d}'.format(dsout.time.values[0],dsout.time.values[-1]))
    # fout.append('{:04d}{:02d}-{:04d}{:02d}'.format(dsout.time.dt.year[0].values,dsout.time.dt.month[0].values,dsout.time.dt.year[-1].values,dsout.time.dt.month[-1].values))
    fout.append('nc')
    fout = '.'.join(fout)
    fout = os.path.join(outdir,fout)

    dsout.to_netcdf(fout,unlimited_dims='time')
    print("wrote {} at {}".format(fout, time.ctime()))

starting loop over 2006-2006 at Mon Mar  6 16:37:03 2023
determining time subset of files to operate on
calculating sigma2_T: Mon Mar  6 16:37:03 2023
calculated sigma2_T: Mon Mar  6 16:37:13 2023
sigma2_T <xarray.DataArray 'density' (time: 12, z_t: 62, nlat: 2400, nlon: 3600)>
dask.array<sub, shape=(12, 62, 2400, 3600), dtype=float32, chunksize=(1, 62, 2400, 100), chunktype=numpy.ndarray>
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.625e+05 5.875e+05
    ULONG    (nlat, nlon) float64 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
    ULAT     (nlat, nlon) float64 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
    TLONG    (nlat, nlon) float64 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
    TLAT     (nlat, nlon) float64 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
  * time     (time) object 2006-01-15 00:00:00 ... 2006-12-15 00:00:00
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:  Sigma referenced to 2000m
    units:      kg/m^3
sigma_mid <xarray.

CancelledError: [('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 1, 0, 13, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 3, 0, 22, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 2, 0, 20, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 5, 0, 31, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 3, 0, 0, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 9, 0, 27, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 6, 0, 3, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 9, 0, 16, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 6, 0, 35, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 5, 0, 20, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 9, 0, 24, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 1, 0, 2, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 6, 0, 11, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 11, 0, 31, 0), ('sum-aggregate-d859386ff4f0892813e53815d72faaf0', 9, 0, 28, 0), ('sum-aggr